In [1]:
import sqlite3
import pandas as pd
from pandas import Timestamp, Period
from typing import Dict, Tuple
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import json

In [2]:
def connect_to_db(db_path: str) -> sqlite3.Connection:
    """Efficiently connect to SQLite database"""
    return sqlite3.connect(db_path, detect_types=sqlite3.PARSE_DECLTYPES)

def preprocess_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """Central datetime preprocessing to reduce redundant operations"""
    datetime_columns = ['创建时间', '预定开始时间', '预定结束时间']
    for col in datetime_columns:
        if col in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def calculate_user_intervals(orders_df: pd.DataFrame) -> pd.DataFrame:
    """计算用户订单间隔"""
    orders_df['上次下单日期'] = orders_df.groupby('手机号')['创建时间'].shift(1)
    orders_df['order_interval'] = (orders_df['创建时间'] - orders_df['上次下单日期']).dt.days
    return orders_df

def convert_keys_to_str(data):
    if isinstance(data, dict):
        return {str(k) if isinstance(k, tuple) else k: convert_keys_to_str(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_keys_to_str(item) for item in data]
    return data

def convert_df_to_dict(data):
    if isinstance(data, pd.DataFrame):
        df = data.copy()
        # 处理时间类型列
        for col in df.columns:
            if pd.api.types.is_datetime64_any_dtype(df[col]) or pd.api.types.is_period_dtype(df[col]):
                df[col] = df[col].astype(str)
        # 处理时间类型的索引
        if pd.api.types.is_datetime64_any_dtype(df.index) or pd.api.types.is_period_dtype(df.index):
            df.index = df.index.astype(str)
        # 将 NaN 替换为 None，确保 JSON 中为 null
        return df.replace({np.nan: None}).to_dict(orient='records')
    elif isinstance(data, dict):
        return {k: convert_df_to_dict(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [convert_df_to_dict(item) for item in data]
    elif isinstance(data, (pd.Timestamp, pd.Period)) or pd.isna(data):
        return str(data) if not pd.isna(data) else None
    return data

In [3]:
def get_db_connection():
    conn = sqlite3.connect('db/ideapod.db')
    conn.row_factory = sqlite3.Row
    return conn

In [14]:
def analyze(conn):

        space_df = pd.read_sql_query("SELECT * FROM Space", conn)
        member_df = pd.read_sql_query("SELECT 会员号, 手机号, 等级 FROM Member", conn)
        
        space_df.set_index("手机号", inplace=True)
        space_df.index = space_df.index.astype(str)
        
    
        # 合并
        space_df = space_df.merge(member_df, 
                                   left_index=True, 
                                   right_on='手机号', 
                                   how='left')

        space_df['等级'] = space_df['等级'].fillna("未注册用户")        
        space_df = space_df[space_df['等级'] != 'ideapod']

        space_df['订单商品名'] = space_df['订单商品名'].fillna('').str.strip().str.replace('上海洛克外滩店-', '').str.replace('ideaPod 二楼专注-', '').str.replace(' the Box', '')
        # 名字修改对应关系需要确认
        space_df['订单商品名'] = space_df['订单商品名'].replace({'丛林心流舱·日':'心流舱·巴赫','丛林心流舱·月':'心流舱·荣格','丛林心流舱·星':'心流舱·雨果','丛林心流舱·辰 ':'心流舱·牛顿','一层半帘区1':'蘑菇半帘区'})
        space_df['订单商品名'] = space_df['订单商品名'].apply(
            lambda x: '图书馆专注区' if x == '图书馆专注' else x
        )
        # 预处理数据，不填充数值列的 NaN，保留为 None
        space_df = preprocess_datetime(space_df)

        space_df['booking_month'] = space_df['创建时间'].dt.to_period('M')
        space_df['booking_week'] = space_df['创建时间'].dt.to_period('W-MON').apply(lambda x: x.start_time.date())
        space_df['开始使用时刻'] = pd.to_datetime(space_df['预定开始时间'], errors='coerce').dt.hour
        space_df['weekday'] = space_df['创建时间'].dt.day_name()
        weekday_map = {
            'Monday': '周一', 'Tuesday': '周二', 'Wednesday': '周三', 
            'Thursday': '周四', 'Friday': '周五', 'Saturday': '周六', 'Sunday': '周日'
        }
        space_df['weekday'] = space_df['weekday'].map(weekday_map)
        
    
        user_df = space_df[space_df['订单商品名'].str.contains('图书馆')]

        target_date = pd.to_datetime('2024-6-2')
        filtered_df = user_df[user_df['预定开始时间'].dt.date == target_date.date()]

        filtered_df = filtered_df[['订单商品名','预定开始时间','预定结束时间','实付金额','订单编号','会员号','手机号','创建时间','修改时间','booking_week','weekday']]
    

        return filtered_df


In [15]:
space_df = analyze(get_db_connection())

print (space_df)


          订单商品名              预定开始时间              预定结束时间  实付金额  \
15143.0  图书馆专注区 2024-06-02 17:54:00 2024-06-02 19:54:00   NaN   
16028.0  图书馆专注区 2024-06-02 16:13:00 2024-06-02 20:13:00  73.0   
16542.0  图书馆专注区 2024-06-02 16:13:00 2024-06-02 20:13:00  48.0   
16015.0  图书馆专注区 2024-06-02 16:04:00 2024-06-02 19:04:00  60.0   
16718.0  图书馆专注区 2024-06-02 15:58:00 2024-06-02 20:28:00  75.5   
16009.0  图书馆专注区 2024-06-02 15:42:00 2024-06-02 18:12:00  37.5   
16066.0  图书馆专注区 2024-06-02 15:13:00 2024-06-02 18:13:00  45.0   
16614.0  图书馆专注区 2024-06-02 14:30:00 2024-06-02 16:30:00  30.0   
16042.0  图书馆专注区 2024-06-02 14:09:00 2024-06-02 18:09:00  60.0   
17116.0  图书馆专注区 2024-06-02 14:09:00 2024-06-02 18:09:00  60.0   
NaN      图书馆专注区 2024-06-02 15:00:00 2024-06-02 17:00:00  45.0   
NaN      图书馆专注区 2024-06-02 13:31:00 2024-06-02 16:01:00  37.5   
16059.0  图书馆专注区 2024-06-02 12:51:00 2024-06-02 14:51:00  45.0   
NaN      图书馆专注区 2024-06-02 14:00:00 2024-06-02 18:00:00  73.0   
NaN      图书馆专注区 2024-06-0

In [23]:
def plot_histogram(data: dict, key: str, column: str, title: str = None):
    df = data[key]  # 从字典中提取 DataFrame
    plt.figure(figsize=(10, 6))
    plt.hist(df[column], bins=30, edgecolor='black')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.title(title if title else f'Histogram of {column}')
    plt.grid(True, alpha=0.3)
    plt.show()


In [4]:
def analyze_member(member_df: pd.DataFrame, db_path: str) -> Dict[str, pd.DataFrame]:
    """合并后的会员分析函数"""
    member_df = calculate_user_intervals(member_df)
    unique_months = sorted(member_df['booking_month'].unique())
    last_order_dates = member_df.groupby('手机号')['创建时间'].max().reset_index()
    first_order_dates = member_df.groupby('手机号')['创建时间'].min().reset_index()
    
    # 初始化结果列表
    monthly_metrics = []
    
    for period_month in unique_months:
        month_start = period_month.to_timestamp()
        month_end = period_month.to_timestamp('M')
        
        # 当前月数据
        current_month_orders = member_df[(member_df['创建时间'] >= month_start) & 
                                       (member_df['创建时间'] <= month_end)]
        active_users = current_month_orders['手机号'].nunique()
        
        # 新增用户
        new_users = len(first_order_dates[(first_order_dates['创建时间'] >= month_start) &
                                        (first_order_dates['创建时间'] <= month_end)]['手机号'].unique())
        
        # 流失相关
        churn_30_60 = len(last_order_dates[(last_order_dates['创建时间'] < month_end - timedelta(days=30)) &
                                         (last_order_dates['创建时间'] >= month_end - timedelta(days=60))]['手机号'].unique())
        churn_60_90 = len(last_order_dates[(last_order_dates['创建时间'] < month_end - timedelta(days=60)) &
                                         (last_order_dates['创建时间'] >= month_end - timedelta(days=90))]['手机号'].unique())
        churn_long = len(last_order_dates[last_order_dates['创建时间'] < month_end - timedelta(days=120)]['手机号'].unique())
        
        # 重新激活用户
        prev_users = member_df[(member_df['创建时间'] < month_start - timedelta(days=90)) &
                             (member_df['创建时间'] >= month_start - timedelta(days=180))]['手机号'].unique()
        reactivated = len(current_month_orders[current_month_orders['手机号'].isin(prev_users)]['手机号'].unique())
        
        # 计算回购用户
        past_users = member_df[member_df['创建时间'] < month_start]['手机号'].unique()
        current_orders = current_month_orders[current_month_orders['手机号'].isin(past_users)].sort_values('创建时间')
        repurchase_users = len(current_orders.groupby('手机号').filter(
            lambda x: (x['创建时间'].max() - x['创建时间'].min()).days >= 1
        )['手机号'].unique())
        
        monthly_metrics.append({
            'booking_month': str(period_month),
            '活跃用户': active_users,
            '新增用户': new_users,
            '流失预警_30_60天': churn_30_60,
            '流失预警_60_90天': churn_60_90,
            '长期未活跃_120天以上': churn_long,
            '重新激活用户': reactivated,
            '回购用户': repurchase_users
        })
    
    counts_df = pd.DataFrame(monthly_metrics)
    
    # 计算按月和会员等级的统计
    monthly_level_stats = member_df.groupby(['booking_month', '等级']).agg({
        '实付金额': ['mean', 'sum'],
        '订单编号': 'count',
        '手机号': 'nunique'
    }).reset_index()
    monthly_level_stats.columns = ['月份', '等级', '平均收入', '总收入', '订单量', '独立会员数量']
    
    # 生成统计表
    avg_revenue_table = pd.pivot_table(monthly_level_stats, values='平均收入', index='月份', columns='等级', fill_value=0).reset_index()
    total_revenue_table = pd.pivot_table(monthly_level_stats, values='总收入', index='月份', columns='等级', fill_value=0).reset_index()
    order_volume_table = pd.pivot_table(monthly_level_stats, values='订单量', index='月份', columns='等级', fill_value=0).reset_index()
    
    
    # 计算一个月留存率、回购率及流失率
    rates_df = pd.DataFrame({'booking_month': counts_df['booking_month']})
    rates_df['一个月留存率'] = 0.0
    rates_df['回购率'] = counts_df['回购用户'] / counts_df['活跃用户'] * 100
    rates_df['30天流失率'] = counts_df['流失预警_30_60天'] / counts_df['活跃用户'].shift(1) * 100
    rates_df['60天流失率'] = counts_df['流失预警_60_90天'] / counts_df['流失预警_30_60天'].shift(1) * 100
    
    for i, month in enumerate(unique_months[1:], 1):
        prev_month = unique_months[i-1]
        prev_new_users = member_df[(member_df['booking_month'] == prev_month) & 
                                 (member_df['手机号'].isin(first_order_dates[
                                     (first_order_dates['创建时间'] >= prev_month.to_timestamp()) &
                                     (first_order_dates['创建时间'] <= prev_month.to_timestamp('M'))
                                 ]['手机号']))]['手机号'].unique()
        current_active = member_df[(member_df['booking_month'] == month) & 
                                 (member_df['手机号'].isin(prev_new_users))]['手机号'].nunique()
        rates_df.loc[i, '一个月留存率'] = current_active / len(prev_new_users) * 100 if len(prev_new_users) > 0 else 0
    
    # 处理NaN值
    counts_df = counts_df.fillna(0)
    rates_df = rates_df.fillna(0)
    

    df_sorted = monthly_level_stats
    #.sort_values(by='总消费金额', ascending=False)
    
    return df_sorted
    